## Preprocessing

In [ ]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 8.3 MB/s 
     |████████████████████████████████| 1.6 MB 51.2 MB/s 


In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
# import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
# application_df = pd.read_csv("../Resources/charity_data.csv")
application_df = pd.read_csv("/content/drive/MyDrive/21-Deep-Learning/Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
application_df.isnull().value_counts()

EIN    NAME   APPLICATION_TYPE  AFFILIATION  CLASSIFICATION  USE_CASE  ORGANIZATION  STATUS  INCOME_AMT  SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL
False  False  False             False        False           False     False         False   False       False                   False    False            34299
dtype: int64

In [ ]:
application_df.shape

(34299, 12)

### **The Third Attempt to Optimize the Model**
#### Drop the non-beneficial ID columns, 'EIN','NAME'and some other columns,'SPECIAL_CONSIDERATIONS','STATUS','USE_CASE'
#### the max number of epochs to 20
#### APPLICATION_TYPE cutoff=600
#### CLASSIFICATION cutoff=287
#### Allow kerastuner to decide which activation function to use in hidden layers: activation = hp.Choice('activation',['relu','tanh'])
#### Allow Kerastuner to decide number of hiden layers and neurons in hidden layers
#### Run the kerasturner search for best hyperparameters
#### Get and evaluate the top 3 models against the test dataset

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df1=application_df.drop(["EIN","NAME","SPECIAL_CONSIDERATIONS","STATUS","USE_CASE"],axis=1)
application_df1.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,0,5000,1
1,T3,Independent,C2000,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,Association,0,5000,0
3,T3,CompanySponsored,C2000,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Trust,100000-499999,142590,1


In [ ]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df1.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
app_type2=application_df1["APPLICATION_TYPE"].value_counts()
app_type2

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace=list(app_type2[app_type2<600].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df1['APPLICATION_TYPE'] = application_df1['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df1['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
classification2=application_df1["CLASSIFICATION"].value_counts()
classification2

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
classifications_to_replace=list(classification2[classification2<287].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df1['CLASSIFICATION'] = application_df1['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df1['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [ ]:
from pandas.core.arrays import numeric
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
numeric_df2=pd.get_dummies(application_df1)
numeric_df2.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [ ]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y1=numeric_df2.IS_SUCCESSFUL.values
X1=numeric_df2.drop(columns="IS_SUCCESSFUL").values
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=42, stratify=y1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X1_scaler = scaler.fit(X1_train)

# Scale the data
X1_train_scaled = X1_scaler.transform(X1_train)
X1_test_scaled = X1_scaler.transform(X1_test)

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options

number_input_features1=len(X1_train_scaled[0])
def create_model(hp):
    nn_model1 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model1.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=number_input_features1))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model1.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model1.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model1

In [ ]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X1_train_scaled,y1_train,epochs=20,validation_data=(X1_test_scaled,y1_test))

Trial 60 Complete [00h 00m 48s]
val_accuracy: 0.7195335030555725

Best val_accuracy So Far: 0.7274635434150696
Total elapsed time: 00h 22m 05s


In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 3,
 'num_layers': 2,
 'units_0': 3,
 'units_1': 9,
 'units_2': 5,
 'units_3': 5,
 'units_4': 1,
 'units_5': 7,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0052'}

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X1_test_scaled,y1_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5781 - accuracy: 0.7275 - 551ms/epoch - 2ms/step
Loss: 0.5780916810035706, Accuracy: 0.7274635434150696


In [ ]:
best_model.save('/content/drive/MyDrive/21-Deep-Learning/AlphabetSoupCharityOptimization3.h5')